# **Cleaning not-null values**

In [48]:
import numpy as np
import pandas as pd
import re

In [2]:
df=pd.DataFrame({
    'Sex':['M','F','M','D','?'],
    'Age':[29,30,24,290,25]
})

df

,Sex,Age
0,M,29
1,F,30
2,M,24
3,D,290
4,?,25


The dataframe **df** doesn't have null values but clearly has invalid values. In age `290` doesn't seem like a valid age and `D` and `?` don't correspond with any known sex category.

In [3]:
df['Sex'].unique()

array(['M', 'F', 'D', '?'], dtype=object)

In [4]:
df['Sex'].value_counts()

Sex
M    2
F    1
D    1
?    1
Name: count, dtype: int64

Lets consider `D` was a typo error and was intended to be `F` now lets replace the values using `replace()` method:

In [5]:
df['Sex'].replace('D','F')

0    M
1    F
2    M
3    F
4    ?
Name: Sex, dtype: object

It can accepts a dictionary of values to replace. For example `M`s were meant to be `N`s

In [6]:
df['Sex'].replace({'D':'F','M':'N'})

0    N
1    F
2    N
3    F
4    ?
Name: Sex, dtype: object

If you have many columns to replace, you could apply it at "DataFrame level":

In [7]:
df.replace({
    'Sex':{
        'D':'F'
    },
    'Age':{
        290:29
    }

})

,Sex,Age
0,M,29
1,F,30
2,M,24
3,F,29
4,?,25


In the previous example, I explicitly replaced 290 with 29(assumint it was just an extra 0 entered at data-entry phase). But what if you'd like to remove all the extra 0s from the ages columns? (example, `150>15`,`490>49`).

The first set would be to just set the limit of the "not possible" age. Is it 100? 120? Let's say that anything above 100 isn't credible for our dataset. We can then combine boolean selection with the operation:

In [8]:
df[df['Age']>100]

,Sex,Age
3,D,290


In [9]:
df.loc[df['Age']>100, 'Age']

3    290
Name: Age, dtype: int64

In [10]:
df.loc[df['Age']>100,'Age']=df.loc[df['Age']>100,'Age']/10

In [11]:
df

,Sex,Age
0,M,29
1,F,30
2,M,24
3,D,29
4,?,25


### **Duplicates**
Checking duplicate values is extremely simple. It'll behave differently between Series and DataFrames. let's start with Series. As an example, let's say we're throwing a fancy party and we;re inviditn Ambassadors from Europe. But can only invite one ambassador per country. This is our original list, and as you can see, both the UK and Germany have duplicated ambassadors:

In [12]:
ambassadors=pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'Gérard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])

In [13]:
ambassadors

Gérard Araud                  France
Kim Darroch           United Kingdom
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
Peter Ammon                  Germany
Klaus Scharioth              Germany
dtype: object

The two most important methods to deal with duplicates are `duplicated`(that will tell you which values are duplicates) and drop_duplicates(which will just get rid of duplicate):

In [14]:
ambassadors.duplicated()

Gérard Araud          False
Kim Darroch           False
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig          False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In this case `duplicated()` didn't consider `Kim Darroch` the first instance of the United Kingdom or `Peter Wittig` as duplicates because by default, it'll consider the first occurence of the value as not-duplicate. You can change this behaviour with the keep parameter:

In [15]:
ambassadors.duplicated(keep='last')

Gérard Araud          False
Kim Darroch            True
Peter Westmacott      False
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth       False
dtype: bool

A similar method is `drop_duplicates()`, which just excludes the duplicated values and also accepts the `keep` parameter:

In [16]:
ambassadors.drop_duplicates()

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
dtype: object

In [17]:
ambassadors.drop_duplicates(keep='last')

Gérard Araud                  France
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

If we pass `False` in the `keep` parameter then all the duplicated value are considered.

In [18]:
ambassadors.duplicated(keep=False)

Gérard Araud          False
Kim Darroch            True
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In [19]:
ambassadors.drop_duplicates(keep=False)

Gérard Araud          France
Armando Varricchio     Italy
dtype: object

### **Duplicates in DataFrames**

Conceptually speaking duplicates in DataFrame happen at row level. Two rows with exactly the same values are considered to be duplicates:

In [20]:
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'LeBron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant',
    ],
    'Pos': [
        'SG',
        'SF',
        'SG',
        'SF',
        'SF'
    ]
})

In [21]:
players

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


Here in the `players` dataframe, we can clearly see that kobe is duplicated but he appears with two different positons. What does `duplicated` say?

In [22]:
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

we can clearly see that all the column values should be duplicates to be considered by `duplicated()`. We can customize this with the `subset` parameter:

In [23]:
players.duplicated(subset=['Pos'])

0    False
1    False
2     True
3     True
4     True
dtype: bool

In [24]:
players.duplicated(subset=['Name'])

0    False
1    False
2     True
3    False
4     True
dtype: bool

We can use the `keep` parameter as above:

In [25]:
players.duplicated(subset=['Name'], keep='last')

0     True
1    False
2     True
3    False
4    False
dtype: bool

In [26]:
players.duplicated(subset=['Pos'],keep=False)

0    True
1    True
2    True
3    True
4    True
dtype: bool

`drop_duplicates()` takes the same parameter

In [27]:
players.drop_duplicates()

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [28]:
players.drop_duplicates(keep=False)

,Name,Pos
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [29]:
players.drop_duplicates(subset=['Name'],keep='last')

,Name,Pos
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


### **Text Handling**

#### **Splitting Columns**

In [30]:
df = pd.DataFrame({
    'Data': [
        '1987_M_US _1',
        '1990?_M_UK_1',
        '1992_F_US_2',
        '1970?_M_   IT_1',
        '1985_F_I  T_2'
]})

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes


In [32]:
df

,Data
0,1987_M_US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


In [33]:
df['Data'].str.split('_')

0       [1987, M, US , 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [34]:
df['Data'].str.split('_', expand=True)

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [35]:
df=df['Data'].str.split('_',expand=True)

In [36]:
df

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [37]:
df.columns=['Year','Sex','Country','No_Children']

In [38]:
df

,Year,Sex,Country,No_Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [39]:
df['Year'].str.contains('\?')

0    False
1     True
2    False
3     True
4    False
Name: Year, dtype: bool

You can also check which columns contain a given value wiht the `contains` method. `contains` take a regex/pattern as first value, so we need to use escape character before`?` symbol as it has a special meaning for these patterns. Regular letters don't need escaping:

In [40]:
df['Country'].str.contains('U')

0     True
1     True
2     True
3    False
4    False
Name: Country, dtype: bool

removing whitespaces (like in `'US '` or `' I T'`) can be achieved with `strip` or just `replace`:

In [45]:
df['Country'].str.strip()

0      US
1      UK
2      US
3      IT
4    I  T
Name: Country, dtype: object

In [42]:
df['Country'].str.replace(' ','')

0    US
1    UK
2    US
3    IT
4    IT
Name: Country, dtype: object

`replace` and `contains` take regex patterns, which can make it easier to replace values in bulk:

In [50]:
df['Year'].str.replace(r'(?P<year>\d{4})\?', lambda m: m.group('year'),regex=True)

0    1987
1    1990
2    1992
3    1970
4    1985
Name: Year, dtype: object